### Setup 

In this section we define our target trial estimands for two scenarios:
- **Per-protocol (PP):** Focused on patients adhering strictly to the treatment protocol.
- **Intention-to-treat (ITT):** Analyses based on the treatment as assigned regardless of adherence.

We also create directories using Python’s `tempfile` module to store model outputs or intermediate files for later inspection.

In [11]:
import os
import pandas as pd
import tempfile

# Define the estimands
estimand_pp = "PP"  # Per-protocol
estimand_itt = "ITT"  # Intention-to-treat

# Create directories to save files for later inspection
trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)

trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

# The data_censored.csv file will be used later for analysis.

In [12]:

# Step 2: Clustering before assigning treatments
from sklearn.cluster import KMeans
import pandas as pd

# Load data
df = pd.read_csv('Data/data_censored.csv')

# Select features for clustering (excluding non-numeric or ID columns)
features = df[['x1', 'x2', 'x3', 'x4', 'age']]

# Apply K-Means clustering
num_clusters = 3  # You can change this based on analysis
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['Cluster_ID'] = kmeans.fit_predict(features)

# Save modified dataset with Cluster_ID
df.to_csv('data_clustered.csv', index=False)


In [13]:

# Step 5: Analyze treatment effects within each cluster
import statsmodels.api as sm

# Load clustered data
df = pd.read_csv('data_clustered.csv')

# Define function to estimate treatment effect per cluster
def analyze_cluster_treatment_effect(df, cluster_id):
    cluster_df = df[df['Cluster_ID'] == cluster_id]
    
    # Define treatment and outcome variables (modify based on dataset structure)
    X = sm.add_constant(cluster_df[['treatment']])  # Predictor: treatment
    y = cluster_df['outcome']  # Outcome variable (modify accordingly)
    
    model = sm.OLS(y, X).fit()
    print(f"Cluster {cluster_id}: Treatment Effect Estimate")
    print(model.summary())
    print("="*60)

# Apply function to each cluster
for cluster in df['Cluster_ID'].unique():
    analyze_cluster_treatment_effect(df, cluster)


Cluster 1: Treatment Effect Estimate
                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                   0.01720
Date:                Sun, 09 Mar 2025   Prob (F-statistic):              0.896
Time:                        19:56:47   Log-Likelihood:                 128.34
No. Observations:                 192   AIC:                            -252.7
Df Residuals:                     190   BIC:                            -246.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          

### Data Preparation

In this section we load the observational data from the `data_censored.csv` file which will be used for the target trial emulation. 
The dataset includes columns such as `id`, `period`, `treatment`, `x1`, `x2`, `x3`, `x4`, `age`, `age_s`, `outcome`, `censored`, and `eligible`.

We then define a helper function `set_data` to associate specific columns with their roles in the trial data. 

For the Per-protocol analysis, the dataset is assigned to the `trial_pp` object using a pipe-like style, while for the ITT analysis, a standard function call is used.

In [14]:
# Load the observational data
data_censored = pd.read_csv('C:\\Github\\Assignment_2_Data_Analytics\\Assignment_1_Clustering_Data_Analytics\\Data\\data_censored.csv')
print(data_censored.head())  # display first few rows


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Github\\Assignment_2_Data_Analytics\\Assignment_1_Clustering_Data_Analytics\\Data\\data_censored.csv'

In [15]:
# Define Per-protocol (PP) dataset with data included
trial_pp = {
    "data": data_censored,
    "id": "id",
    "period": "period",
    "treatment": "treatment",
    "outcome": "outcome",
    "eligible": "eligible"
}

# Define Intention-to-Treat (ITT) dataset with data included
trial_itt = {
    "data": data_censored,
    "id": "id",
    "period": "period",
    "treatment": "treatment",
    "outcome": "outcome",
    "eligible": "eligible"
}

# Compute total observations and unique patients
n_obs = len(data_censored)
n_patients = data_censored['id'].nunique()

# Get the first 2 rows and last 2 rows of the data
head_df = data_censored.head(2)
tail_df = data_censored.tail(2)

def print_data_showcase(data, estimand_label):
    # Compute total observations and unique patients
    n_obs = len(data)
    n_patients = data['id'].nunique()
    
    # Get the first 2 rows and last 2 rows of the data
    head_df = data.head(2)
    tail_df = data.tail(2)
    
    # Manually construct header strings with column names and types (as in provided example)
    print("Trial Sequence Object")
    print("Estimand: " + estimand_label)
    print("\nData:")
    print("  - N: {} observations from {} patients".format(n_obs, n_patients))
    print("         id period treatment    x1           x2   x3        x4   age      age_s")
    print("      <int> <int>     <num> <num>        <num> <int>     <num> <num>      <num>")
    print(head_df.to_string(index=True))
    print("---")
    print(tail_df.to_string(index=True))
    # For the outcome part, print outcome, censored, eligible columns similarly:
    print("\n      outcome censored eligible")
    print("        <num>    <int>    <num>")
    head_outcome = head_df[["outcome", "censored", "eligible"]]
    tail_outcome = tail_df[["outcome", "censored", "eligible"]]
    print(head_outcome.to_string(index=True))
    print("---")
    print(tail_outcome.to_string(index=True))
    print("\n" + "-"*80 + "\n")

# Print showcase for Per-protocol (PP) trial
print_data_showcase(trial_pp["data"], "Per-protocol")

# Print showcase for Intention-to-treat (ITT) trial
print_data_showcase(trial_itt["data"], "Intention-to-treat")


NameError: name 'data_censored' is not defined

## Weight Models and Censoring

In this step we adjust for informative censoring by applying inverse probability of censoring weights (IPCW). Time-to-event models are constructed to estimate the probability that an observation is not censored, and these probabilities are later used to compute stabilized weights. The configuration of these weight models is stored in the trial objects, while the actual model fitting is deferred until a function such as `calculate_weights()` is invoked.

- **Censoring Due to Treatment Switching (PP only):**  
  For the Per-protocol estimand, separate models are specified for the numerator (using a limited set of covariates such as age) and the denominator (using an extended set like age, x1, and x3). A dummy model fitter, simulating logistic regression, is used to configure the weight models without immediately fitting them.

- **Other Informative Censoring:**  
  For both PP and ITT, models are defined to estimate the probability of remaining uncensored. This involves specifying the censoring event (e.g., the "censored" column) along with numerator and denominator models (e.g., using x2 in the numerator vs. x2 + x1 in the denominator) and an option to pool models. The configurations are stored, and the models are fit later when needed.

In [16]:
# Define a dummy model fitter to simulate fitting using logistic regression
class StatsGLMLogit:
    def __init__(self, save_path):
        self.save_path = save_path
    def __repr__(self):
        return f"te_stats_glm_logit (save_path={self.save_path})"

# Function to set switch weight model (used only for PP)
def set_switch_weight_model(trial, numerator, denominator, model_fitter):
    trial["switch_weights_config"] = {
         "numerator_formula": f"treatment ~ {numerator}",
         "denominator_formula": f"treatment ~ {denominator}",
         "model_fitter": model_fitter,
         "note": "Weight models not fitted. Use calculate_weights()"
    }
    return trial

# Function to set censor weight model for informative censoring
def set_censor_weight_model(trial, censor_event, numerator, denominator, pool_models, model_fitter):
    trial["censor_weights_config"] = {
         "censor_event": censor_event,
         "numerator_formula": f"1 - {censor_event} ~ {numerator}",
         "denominator_formula": f"1 - {censor_event} ~ {denominator}",
         "pool_models": pool_models,
         "model_fitter": model_fitter,
         "note": "Weight models not fitted. Use calculate_weights()"
    }
    return trial

# Apply treatment switching weight model for PP and assign to a distinct variable.
trial_pp_switch = set_switch_weight_model(
    trial_pp,
    numerator="age",
    denominator="age + x1 + x3",
    model_fitter=StatsGLMLogit(save_path=os.path.join(trial_pp_dir, "switch_models"))
)
print("trial_pp_switch weights config (treatment switching):")
print(trial_pp_switch["switch_weights_config"])

# Apply censor weight model on a copy of PP to keep it separate.
trial_pp_censor = set_censor_weight_model(
    trial_pp.copy(),
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="none",
    model_fitter=StatsGLMLogit(save_path=os.path.join(trial_pp_dir, "censor_models"))
)
print("trial_pp_censor weights config (censoring):")
print(trial_pp_censor["censor_weights_config"])

# For ITT, censoring weights remain as before.
trial_itt = set_censor_weight_model(
    trial_itt,
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="numerator",
    model_fitter=StatsGLMLogit(save_path=os.path.join(trial_itt_dir, "censor_models"))
)
print("trial_itt censor_weights_config:")
print(trial_itt["censor_weights_config"])


NameError: name 'trial_pp' is not defined

## Calculate Weights

In this step we fit the individual models that were configured in Step 3 and then combine them into inverse probability of censoring weights (IPCW). The function `calculate_weights()` is used to perform the model fitting. The fitted model objects are saved on disk in the directories we created earlier, and the weight model summaries are stored in the trial sequence objects.

In [17]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import joblib

def calculate_itt_weights(trial):
    # Work with the full dataset (725 observations)
    data = trial["data"].copy()
    data = data.sort_values(["id", "period"])
    data["prev_treatment"] = data.groupby("id")["treatment"].shift(1).fillna(0)
    data["not_censored"] = 1 - data["censored"]
    
    # Model n: P(censor_event = 0 | X)
    formula_n = "not_censored ~ x2"
    model_n = smf.logit(formula=formula_n, data=data).fit(disp=0)
    trial["fitted_itt_censor_numerator"] = model_n
    
    # Model d0: P(censor_event = 0 | X, previous treatment = 0)
    subset_d0 = data[data["prev_treatment"] == 0]
    formula_d = "not_censored ~ x2 + x1"
    model_d0 = smf.logit(formula=formula_d, data=subset_d0).fit(disp=0)
    trial["fitted_itt_censor_denominator_d0"] = model_d0
    
    # Model d1: P(censor_event = 0 | X, previous treatment = 1)
    subset_d1 = data[data["prev_treatment"] == 1]
    model_d1 = smf.logit(formula=formula_d, data=subset_d1).fit(disp=0)
    trial["fitted_itt_censor_denominator_d1"] = model_d1
    
    # Compute predicted probabilities for all observations
    data["pred_num"] = model_n.predict(data)
    data["pred_den"] = 0.0
    idx0 = data["prev_treatment"] == 0
    idx1 = data["prev_treatment"] == 1
    data.loc[idx0, "pred_den"] = model_d0.predict(data.loc[idx0])
    data.loc[idx1, "pred_den"] = model_d1.predict(data.loc[idx1])
    
    # Calculate weight as the ratio of predicted numerator to denominator
    data["weight"] = data["pred_num"] / data["pred_den"]
    
    # Save the full dataset with calculated weights
    trial["data_with_weights"] = data.copy()
    
    return trial

# --- Compute ITT weights ---
trial_itt = calculate_itt_weights(trial_itt)

# --- Define the path to save the CSV file ---
data_folder = r"C:\Github\Assignment_2_Data_Analytics\Assignment_1_Clustering_Data_Analytics\Data"
os.makedirs(data_folder, exist_ok=True)
csv_path = os.path.join(data_folder, "trial_itt_data_with_weights.csv")

# --- Save the data with weights ---
trial_itt["data_with_weights"].to_csv(csv_path, index=False)
print("Stored ITT data with calculated weights to CSV file at:")
print(csv_path)

# --- Print the fitted models' summaries ---
print("\nModel n (Numerator) Summary:")
print(trial_itt["fitted_itt_censor_numerator"].summary2().as_text())

print("\nModel d0 (Denom. for prev_treatment = 0) Summary:")
print(trial_itt["fitted_itt_censor_denominator_d0"].summary2().as_text())

print("\nModel d1 (Denom. for prev_treatment = 1) Summary:")
print(trial_itt["fitted_itt_censor_denominator_d1"].summary2().as_text())


NameError: name 'trial_itt' is not defined

In [18]:
import os
import pandas as pd
import statsmodels.formula.api as smf
import joblib

def calculate_pp_informative_weights_updated(trial):
    data = trial["data"].copy()
    data = data.sort_values(["id", "period"])
    data["prev_treatment"] = data.groupby("id")["treatment"].shift(1).fillna(0)
    data["not_censored"] = 1 - data["censored"]
    
    # Model n0: P(censor_event = 0 | X, previous treatment = 0) for numerator
    subset0 = data[data["prev_treatment"] == 0]
    model_n0 = smf.logit("not_censored ~ x2", data=subset0).fit(disp=0)
    trial["fitted_pp_censor_numerator_n0"] = model_n0

    # Model n1: P(censor_event = 0 | X, previous treatment = 1) for numerator
    subset1 = data[data["prev_treatment"] == 1]
    model_n1 = smf.logit("not_censored ~ x2", data=subset1).fit(disp=0)
    trial["fitted_pp_censor_numerator_n1"] = model_n1

    # Model d0: P(censor_event = 0 | X, previous treatment = 0) for denominator
    model_d0 = smf.logit("not_censored ~ x2 + x1", data=subset0).fit(disp=0)
    trial["fitted_pp_censor_denominator_d0"] = model_d0

    # Model d1: P(censor_event = 0 | X, previous treatment = 1) for denominator
    model_d1 = smf.logit("not_censored ~ x2 + x1", data=subset1).fit(disp=0)
    trial["fitted_pp_censor_denominator_d1"] = model_d1

    # Compute predicted probabilities
    data["pred_num"] = 0.0
    data["pred_den"] = 0.0
    idx0 = data["prev_treatment"] == 0
    idx1 = data["prev_treatment"] == 1

    data.loc[idx0, "pred_num"] = model_n0.predict(data.loc[idx0])
    data.loc[idx1, "pred_num"] = model_n1.predict(data.loc[idx1])
    data.loc[idx0, "pred_den"] = model_d0.predict(data.loc[idx0])
    data.loc[idx1, "pred_den"] = model_d1.predict(data.loc[idx1])

    # Calculate weight as the ratio of numerator to denominator predictions
    data["weight"] = data["pred_num"] / data["pred_den"]

    # Save the full dataset with calculated weights
    trial["data_with_weights"] = data.copy()

    return trial

# --- Compute PP weights ---
trial_pp_censor["estimand"] = "PP"
trial_pp_censor["save_dir"] = os.path.join(trial_pp_dir, "informative_censor_models")
os.makedirs(trial_pp_censor["save_dir"], exist_ok=True)
trial_pp_censor = calculate_pp_informative_weights_updated(trial_pp_censor)

# --- Define the path to save the CSV file ---
data_folder = r"C:\Github\Assignment_2_Data_Analytics\Assignment_1_Clustering_Data_Analytics\Data"
os.makedirs(data_folder, exist_ok=True)
csv_path = os.path.join(data_folder, "trial_pp_data_with_weights.csv")

# --- Save the data with weights ---
trial_pp_censor["data_with_weights"].to_csv(csv_path, index=False)
print("Stored PP data with calculated weights to CSV file at:")
print(csv_path)

# --- Print the fitted models' summaries ---
print("\nPP model n0 (Numerator, prev_treatment = 0) Summary:")
print(trial_pp_censor["fitted_pp_censor_numerator_n0"].summary2().as_text())

print("\nPP model n1 (Numerator, prev_treatment = 1) Summary:")
print(trial_pp_censor["fitted_pp_censor_numerator_n1"].summary2().as_text())

print("\nPP model d0 (Denominator, prev_treatment = 0) Summary:")
print(trial_pp_censor["fitted_pp_censor_denominator_d0"].summary2().as_text())

print("\nPP model d1 (Denominator, prev_treatment = 1) Summary:")
print(trial_pp_censor["fitted_pp_censor_denominator_d1"].summary2().as_text())


NameError: name 'trial_pp_censor' is not defined

## Treatment Switching Weight Calculation

In this section, we calculate the treatment switching weights for the Per-protocol (PP) analysis.
We train four logistic regression models:
- model n1: P(treatment = 1 | previous treatment = 1) for numerator.
- model d1: P(treatment = 1 | previous treatment = 1) for denominator.
- model n0: P(treatment = 1 | previous treatment = 0) for numerator.
- model d0: P(treatment = 1 | previous treatment = 0) for denominator.

The weights are calculated as follows:
- For observations with previous treatment = 1:
    weight = (predicted probability from model n1) / (predicted probability from model d1)
- For observations with previous treatment = 0:
    weight = (predicted probability from model n0) / (predicted probability from model d0)

Logistic regression is used to estimate these probabilities and the models are saved to disk.

In [19]:
# New Code Cell: Calculate Treatment Switching Weights using Logistic Regression
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import joblib
import os

def calculate_pp_switch_weights(trial):
    # Ensure trial is PP and prepare data
    data = trial["data"].copy()
    data = data.sort_values(["id", "period"])
    data["prev_treatment"] = data.groupby("id")["treatment"].shift(1).fillna(0)
    
    # Model n1: P(treatment = 1 | previous treatment = 1)
    subset_n1 = data[data["prev_treatment"] == 1]
    formula_n1 = "treatment ~ age"
    model_n1 = smf.logit(formula=formula_n1, data=subset_n1).fit(disp=0)
    save_path_n1 = os.path.join(trial.get("save_dir", ""), "pp_switch_num_model_n1.pkl")
    joblib.dump(model_n1, save_path_n1)
    trial["fitted_pp_switch_numerator_n1"] = model_n1
    
    # Model d1: P(treatment = 1 | previous treatment = 1)
    formula_d1 = "treatment ~ age + x1 + x3"
    model_d1 = smf.logit(formula=formula_d1, data=subset_n1).fit(disp=0)
    save_path_d1 = os.path.join(trial.get("save_dir", ""), "pp_switch_den_model_d1.pkl")
    joblib.dump(model_d1, save_path_d1)
    trial["fitted_pp_switch_denominator_d1"] = model_d1
    
    # Model n0: P(treatment = 1 | previous treatment = 0)
    subset_n0 = data[data["prev_treatment"] == 0]
    formula_n0 = "treatment ~ age"
    model_n0 = smf.logit(formula=formula_n0, data=subset_n0).fit(disp=0)
    save_path_n0 = os.path.join(trial.get("save_dir", ""), "pp_switch_num_model_n0.pkl")
    joblib.dump(model_n0, save_path_n0)
    trial["fitted_pp_switch_numerator_n0"] = model_n0
    
    # Model d0: P(treatment = 1 | previous treatment = 0)
    formula_d0 = "treatment ~ age + x1 + x3"
    model_d0 = smf.logit(formula=formula_d0, data=subset_n0).fit(disp=0)
    save_path_d0 = os.path.join(trial.get("save_dir", ""), "pp_switch_den_model_d0.pkl")
    joblib.dump(model_d0, save_path_d0)
    trial["fitted_pp_switch_denominator_d0"] = model_d0
    
    return trial

# Usage example for trial PP:
# Ensure trial_pp has an assigned save_dir (e.g., within trial_pp_dir)
trial_pp_switch["estimand"] = "PP"
trial_pp_switch["save_dir"] = os.path.join(trial_pp_dir, "switch_models")
os.makedirs(trial_pp_switch["save_dir"], exist_ok=True)
trial_pp_switch = calculate_pp_switch_weights(trial_pp_switch)

# To verify, you can print summaries:
print("Model n1 (Numerator for prev_treatment = 1) Summary:")
print(trial_pp_switch["fitted_pp_switch_numerator_n1"].summary2().as_text())
print("\nModel d1 (Denom. for prev_treatment = 1) Summary:")
print(trial_pp_switch["fitted_pp_switch_denominator_d1"].summary2().as_text())
print("\nModel n0 (Numerator for prev_treatment = 0) Summary:")
print(trial_pp_switch["fitted_pp_switch_numerator_n0"].summary2().as_text())
print("\nModel d0 (Denom. for prev_treatment = 0) Summary:")
print(trial_pp_switch["fitted_pp_switch_denominator_d0"].summary2().as_text())


NameError: name 'trial_pp_switch' is not defined

## Specify Outcome Model
Now we can specify the outcome model. Here we can include adjustment terms for any variables in the dataset. The numerator terms from the stabilised weight models are automatically included in the outcome model formula.

In [20]:
def process_outcome_data(trial, adjustment_terms=""):
    # Use data with weights if available; otherwise, use the original data.
    data = trial.get("data_with_weights", trial["data"]).copy()
    
    # Ensure assigned_treatment exists in the dataset (same as treatment)
    data["assigned_treatment"] = data.get("assigned_treatment", data["treatment"])
    
    # Store adjustment terms in trial dictionary for later use
    trial["processed_data"] = data
    trial["adjustment_terms"] = adjustment_terms
    
    return trial

# --- Process outcome data for PP and ITT using the data that already has calculated weights
trial_pp = process_outcome_data(trial_pp)
trial_itt = process_outcome_data(trial_itt, adjustment_terms=" + x2")

# --- Print data structure summaries
print("PP Processed Data Sample:")
print(trial_pp["processed_data"].head())

print("\nITT Processed Data Sample:")
print(trial_itt["processed_data"].head())


NameError: name 'trial_pp' is not defined

## Expand Trials

We prepare to create the dataset that includes the sequence of target trials. This involves expanding the trial data to include all possible sequences of treatment and control assignments for each patient. 

We use the `set_expansion_options` function to configure the expansion process. This function allows us to specify the output method and the chunk size, which determines the number of patients to include in each expansion iteration. 

For both the Per-protocol (PP) and Intention-to-treat (ITT) analyses, we set the output to a dummy function `save_to_datatable()` and the chunk size to 500 patients.


In [21]:
import os
import pandas as pd
import numpy as np

def expand_trials(data, chunk_size=500):
    """
    Expands trial data in chunks, repeating each row for multiple trial periods.
    """
    expanded_chunks = []

    # Define trial periods (consistent with tutorial defaults)
    periods = np.arange(11)  # Includes periods 0 to 10

    # Process data in chunks
    for start in range(0, len(data), chunk_size):
        chunk = data.iloc[start:start + chunk_size].copy()

        expanded_chunk = (
            chunk.loc[chunk.index.repeat(len(periods))]  # Repeat rows for each period
            .assign(trial_period=np.tile(periods, len(chunk)))  # Assign trial periods
            .reset_index(drop=True)
        )

        # Assign follow-up time (same as trial_period)
        expanded_chunk["followup_time"] = expanded_chunk["trial_period"]

        expanded_chunks.append(expanded_chunk)

    # Concatenate all expanded chunks
    return pd.concat(expanded_chunks, ignore_index=True)

# --- Expand both PP and ITT datasets ---
trial_pp_expanded = expand_trials(trial_pp["processed_data"], chunk_size=500)
trial_itt_expanded = expand_trials(trial_itt["processed_data"], chunk_size=500)

# --- Define the path to save the CSV files ---
data_folder = r"C:\Github\Assignment_2_Data_Analytics\Assignment_1_Clustering_Data_Analytics\Data"
os.makedirs(data_folder, exist_ok=True)  # Ensure the folder exists

# --- Save expanded datasets ---
csv_path_pp = os.path.join(data_folder, "trial_pp_expanded.csv")
csv_path_itt = os.path.join(data_folder, "trial_itt_expanded.csv")

trial_pp_expanded.to_csv(csv_path_pp, index=False)
trial_itt_expanded.to_csv(csv_path_itt, index=False)

print("Stored PP expanded data at:", csv_path_pp)
print("Stored ITT expanded data at:", csv_path_itt)

# --- Check sample output ---
print("\nSample from PP Expanded Data:")
print(trial_pp_expanded.head())


NameError: name 'trial_pp' is not defined

##  Load or Sample from Expanded Data

In [22]:
import pandas as pd
import numpy as np
import os

def load_expanded_data(trial, seed=1234, p_control=0.5, period_range=None):

    np.random.seed(seed)  # Set seed for reproducibility

    # Load the expanded data
    data = trial["expanded_data"].copy()

    # Apply period filtering if specified
    if period_range:
        min_period, max_period = period_range
        data = data[(data["trial_period"] >= min_period) & (data["trial_period"] <= max_period)]

    # Apply p_control sampling: Keep all outcome == 1, sample outcome == 0
    outcome_0_mask = (data["outcome"] == 0)
    sampled_data = data.loc[~outcome_0_mask | (np.random.rand(len(data)) < p_control)]

    return sampled_data

# --- Load and sample from expanded ITT data ---
trial_itt["expanded_data"] = trial_itt_expanded  # Ensure expanded data is stored in trial_itt
sampled_itt_data = load_expanded_data(trial_itt, seed=1234, p_control=0.5)

# --- Define the path to save the sampled data ---
data_folder = r"C:\Github\Assignment_2_Data_Analytics\Assignment_1_Clustering_Data_Analytics\Data"
os.makedirs(data_folder, exist_ok=True)  # Ensure folder exists

csv_path_sampled = os.path.join(data_folder, "trial_itt_load.csv")  # Correct variable name

# --- Save the sampled dataset ---
sampled_itt_data.to_csv(csv_path_sampled, index=False)

print("Stored sampled ITT data at:", csv_path_sampled)

# --- Check sample output ---
print("\nSample from ITT Sampled Data:")
print(sampled_itt_data.head())


NameError: name 'trial_itt_expanded' is not defined

##  Fit Marginal Structural Model

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def fit_msm(data, weight_cols=["weight", "sample_weight"], modify_weights=True):

    # Ensure weights exist
    for col in weight_cols:
        if col not in data.columns:
            data[col] = 1.0  # Default to 1 if missing

    # Combine weights multiplicatively
    data["final_weight"] = data[weight_cols].prod(axis=1)

    # Winsorization: Cap extreme weights at the 99th percentile
    if modify_weights:
        q99 = data["final_weight"].quantile(0.99)
        data["final_weight"] = np.minimum(data["final_weight"], q99)

    # Fit logistic regression model for the outcome (example model)
    formula = "outcome ~ treatment + trial_period + x1 + x2"
    model = sm.GLM.from_formula(formula, data, 
                                family=sm.families.Binomial(), 
                                freq_weights=data["final_weight"]).fit()

    return model

# --- Fit MSM on ITT dataset ---
trial_itt_msm = fit_msm(sampled_itt_data)

# --- Print summary of fitted MSM model ---
print("\nMarginal Structural Model (MSM) Summary:")
print(trial_itt_msm.summary())


NameError: name 'sampled_itt_data' is not defined

## Inference

In [24]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

def predict_survival(model, sampled_itt_data, predict_times=np.arange(0, 11)):
    """
    Predicts survival probabilities for control and treated groups over follow-up time.
    Uses `sampled_itt_data`, which contains follow-up time information.
    
    The model should be a fitted statsmodels regression (e.g., from fit_msm()).
    """

    # Verify required predictors exist.
    required_cols = ["treatment", "followup_time", "x1", "x2"]
    missing_cols = [col for col in required_cols if col not in sampled_itt_data.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns in sampled_itt_data: {missing_cols}")

    # Prepare prediction datasets for control and treated groups.
    data_control = sampled_itt_data.copy()
    data_control["treatment"] = 0  # Control group

    data_treated = sampled_itt_data.copy()
    data_treated["treatment"] = 1  # Treated group

    # Add constant term for prediction.
    predictors = ["treatment", "followup_time", "x1", "x2"]
    data_control_const = sm.add_constant(data_control[predictors])
    data_treated_const = sm.add_constant(data_treated[predictors])

    # Compute predicted risks/probabilities using the fitted model.
    preds_control = model.predict(data_control_const)
    preds_treated = model.predict(data_treated_const)

    # Compute survival probabilities as 1 minus predicted risk.
    survival_control = 1 - preds_control.groupby(data_control["followup_time"]).mean()
    survival_treated = 1 - preds_treated.groupby(data_treated["followup_time"]).mean()

    # Compute survival difference between treated and control.
    survival_diff = survival_treated - survival_control

    # Compute an approximate 95% confidence interval using the normal approximation.
    ci = 1.96 * survival_diff.std()  # Element-wise standard deviation

    # Plot results.
    plt.figure(figsize=(8, 5))
    plt.plot(survival_diff.index, survival_diff, label="Survival Difference", color="blue")
    plt.fill_between(survival_diff.index, survival_diff - ci, survival_diff + ci, 
                     color="red", alpha=0.2, label="95% CI")
    plt.xlabel("Follow-up Time")  # Updated x-axis label
    plt.ylabel("Survival Difference")
    plt.title("Predicted Survival Difference Over Follow-up Time")
    plt.legend()
    plt.show()

# Example usage:
# Assume trial_itt_msm is the fitted marginal structural model produced by fit_msm(),
# and sampled_itt_data is the correctly loaded dataset with follow-up time information.
predict_survival(trial_itt_msm, sampled_itt_data)


NameError: name 'trial_itt_msm' is not defined


### **Insights on TTEv2 - Clustering and Treatment Effects**
1. **Clustering Approach:**
   - K-Means clustering is used with **3 clusters**.
   - Features used for clustering: `x1`, `x2`, `x3`, `x4`, and `age`.
   - The dataset is modified to include a `Cluster_ID` column.

2. **Treatment Effects Across Clusters:**
   - The model estimates treatment effects within each cluster using **OLS regression**.
   - The regression outputs suggest that the **effect of treatment varies across clusters**.
   - Differences in coefficients and significance levels indicate that **some clusters respond better to treatment than others**.

3. **Do Clusters Respond Differently to Treatment?**
   - Yes, the OLS results show **variation in treatment effects** across clusters.
   - Some clusters may have a **stronger or weaker** response to treatment.
   - This suggests that **targeted interventions** may be more effective than a one-size-fits-all approach.
